In [35]:
from empath import Empath
import requests
import json
from nltk.stem import WordNetLemmatizer

In [36]:
def create_category(self,name,seeds,model="fiction",size=100,write=True):
    resp = requests.post(self.backend_url + "/create_category", json={"terms":seeds,"size":size,"model":model})
    results = json.loads(resp.text)
    lemma_words = list()
    lemmatizer = WordNetLemmatizer() 
    for word in results:
        lemma_words.append(lemmatizer.lemmatize(word))
    self.cats[name] = list(set(lemma_words))
    if write:
        with open(self.base_dir+"/data/user/"+name+".empath","w") as f:
            f.write("\t".join([name]+results))

Empath.create_lemma_category = create_category

In [37]:
import nltk
from nltk.corpus import wordnet
lexicon = Empath()
lemmatizer = WordNetLemmatizer()
from stanza.server import CoreNLPClient
from unidecode import unidecode

In [67]:
def create_lexicons(rb,lv,fp,ct):
    lexicon.create_lemma_category("religious_buildings", ["church","mosque", "temple"], model="fiction", size = rb)
    lexicon.create_lemma_category("loc_verbs", ["arrive", "visit", "travel", "return"], model = "fiction", size= lv)
    lexicon.create_lemma_category("fictional_places", ["place","buildings"], model ="fiction", size =fp)
    lexicon.create_lemma_category("custom_times", ["once_upon_a_time", "next_day","that_evening"], size = ct)
    #lexicon.create_lemma_category("custom_times", ["next_day","nex_month","next_year","one_day","once_upon_a_time",
    #                                               "that_evening","sundown","later_that_day","two_hours_later"], size = ct)

In [68]:
create_lexicons(30,14,300,500)

In [69]:
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [70]:
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [71]:
story_names = []
file = open("D:\Jupyter\BTP\Panchatantra\Storynames_new.txt")
file_story_names = file.readlines()
for name in file_story_names:
    story_names.append(name.strip('\n'))
file.close()
print(story_names)

['the_story_of_the_merchant_son', 'the_thief_and_the_brahmins', 'the_monkey_and_the_crocodile', 'the_monkey_the_wedge', 'a_daring_plan', 'Buddha_remains_cool', 'moocha_raja', 'raman_horse_trainer', 'talkative_turtle', 'tenali_outwits_guards', 'tenali_the_detective']


In [72]:
story_names = []
file = open("D:\Jupyter\BTP\Panchatantra\Storynames_new.txt")
file_story_names = file.readlines()
for name in file_story_names:
    story_names.append(name.strip('\n'))
file.close()
print(story_names)

['the_story_of_the_merchant_son', 'the_thief_and_the_brahmins', 'the_monkey_and_the_crocodile', 'the_monkey_the_wedge', 'a_daring_plan', 'Buddha_remains_cool', 'moocha_raja', 'raman_horse_trainer', 'talkative_turtle', 'tenali_outwits_guards', 'tenali_the_detective']


In [73]:
no_splitting = [[4, 14, 15, 16, 21, 22, 30, 46, 47, 52, 53],
                [15, 16, 19, 20, 25, 26, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 45, 46],
                [4, 5, 16, 17, 20, 26, 27, 28, 29, 39, 40, 44, 45, 46, 47, 53, 54, 55, 56, 57],
                [],
                [6, 15, 16, 26, 27, 28],
                [11, 17, 18, 19, 20, 21, 22],
                [],
                [],
                [10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 39],
                [14, 15, 16, 17, 18, 30, 31, 32, 33, 34],
                [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 24, 25, 34, 35, 36, 37, 38],
               ]

no_splitting_alternate = [[4, 5, 14, 15, 16, 17, 21, 22, 23, 30, 31, 46, 47, 48, 52, 53, 54],
[15, 16, 17, 19, 20, 21, 25, 26, 27, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 45, 46, 47],
[4, 5, 6, 16, 17, 18, 20, 21, 26, 27, 28, 29, 30, 39, 40, 41, 44, 45, 46, 47, 48, 53, 54, 55, 56, 57, 58],
[]]

In [74]:
def open_story(Storyname):
    file = open("D:\Jupyter\BTP\Panchatantra\\"+Storyname+'.txt', encoding = 'utf-8')
    text = file.read()
    text = unidecode(text)
    file.close()
    return text
def annotate_story(text):
    with CoreNLPClient(annotators = ['tokenize','ssplit'],
        memory='5G', be_quiet=True, outputFormat = 'json', max_char_length=500000, timeout=36000000) as client:
        annotated_story = client.annotate(text)
    return annotated_story
def open_and_annotate(Storyname):
    text = open_story(Storyname)
    annotated_story = annotate_story(text)
    return text, annotated_story

In [75]:
def events_by_location_and_time(text,ann):
    """
    Non-hierarchy model
    """
    #This function finds sum of dictionary returned by lexicon.analyze i.e., it finds the presence of location words.
    def sum_of_locs_dict(dictionary):
        sum_ = 0
        for key in dictionary.keys():
            sum_ = sum_ + dictionary[key]
        return sum_
       
    lexicon = Empath()   #Part of code used to bring Empath in
    locations_dict = dict()     #Dictionary that holds
    location = "Unknown"    #The variable place will hold latest location word.
                            #It is initilized to "unknown" beacuse till now we haven't encountered any location word.
    loaction_by_sentence = []
    location_to_number = dict() # Convert location words to numbers for better representation
    loc_num = 0 # Will be used to put location words as numbers in the location_to_number dict
    total_sentences = 0
    
    #Take each sentence of the story one by one (ann.sentence returns individual sentences of the story as objects)
    for i, sentence in enumerate(ann.sentence):
        # Remove comma and fullstop beacuse lexicon.analyze cannot identify words if they are followd by a fullstop or comma.
        # text[characterOffsetBegin:characterOffserEnd] is the actual sentence (as a string) of the sentence object returned
        sentence_for_empath = text[sentence.characterOffsetBegin:sentence.characterOffsetEnd].replace(", "," ").replace(".","").replace("-"," ").replace("?","").replace("!","").replace(":"," ")
        #Lemmatize the words you encounter for better identification when being analysed by lexicon.analyze
        #May be commented out because lexicon.create_category does not give good words when singular words are used
        sentence_for_empath = lemmatize_sentence(sentence_for_empath) # Sentences are all lemmatized now
        # Analyze the things
        lexicon_locations_dict = lexicon.analyze(sentence_for_empath,
                                                categories=["religious_buildings", "loc_verbs", "fictional_places", "custom_times"])
        
        s = sum_of_locs_dict(lexicon_locations_dict)
        if s>0:
            words = sentence_for_empath.split(" ")
            # Find if place is same as previous
            for word in words:
                # If the word is a location word
                if sum_of_locs_dict(lexicon.analyze(word,
                                                   categories=["religious_buildings", "loc_verbs", "fictional_places", "custom_times"]))>0:
                    #if new location word encountered is the same as the last location word encountered
                    if word == location:
                        break
                    else:
                        location = word
                        if location in locations_dict:
                            location+="1"
                        locations_dict[location]=[i-1]
                        location_to_number[location] = loc_num
                        loc_num += 1
        else: 
            if location not in locations_dict:
                locations_dict[location] = list()
            locations_dict[location].append(i-1)
        total_sentences = i
    return locations_dict, location_to_number, total_sentences

In [76]:
def construct_event_list(locations_dict, no_split):
    events= []
    for location in locations_dict:
        if locations_dict[location][0] not in no_split:
            events.append(locations_dict[location][0])
    events.sort()
    events.append(total_sentences)
    while events[0] == -1:
        del events[0]
    return set(events)

In [77]:
# def sets_to_lists(my_list_of_e):
#     my_list_of_events_2 = []
#     for i,event in enumerate(my_list_of_e):    
#         x = sorted(list(event))
#         my_list_of_events_2.append(x)
#     print(my_list_of_events_2)

In [78]:
my_list_of_events = []
for i,name in enumerate(story_names):
    text , annotated_story = open_and_annotate(name)
    locations_dict, location_number_map, total_sentences = events_by_location_and_time(text, annotated_story)
    my_list_of_events.append(construct_event_list(locations_dict, no_splitting[i]))

2021-04-18 17:50:27 INFO: Writing properties to tmp file: corenlp_server-c890eba709854eb4.props
2021-04-18 17:50:27 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-c890eba709854eb4.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-04-18 17:50:28 INFO: Writing properties to tmp file: corenlp_server-293735c4bdf54218.props
2021-04-18 17:50:28 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-293735c4bdf54218.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-04-18 17:50:30 INFO: Writing properties to tmp file: corenlp_server-7e17be4cf6a540c1.props
2021-04-18 17:50:30 INFO: 

In [79]:
my_list_of_events

[{0, 5, 6, 7, 9, 26, 29, 32, 45, 50, 61, 64, 65},
 {1, 7, 17, 22, 24, 48},
 {1, 8, 23, 60},
 {0, 1, 3, 11},
 {8, 17, 33},
 {32},
 {1, 16, 22, 37},
 {5, 18, 22},
 {0, 5, 25, 28, 32, 47, 49, 52, 56},
 {35, 37},
 {38}]

In [80]:
my_list_of_events

[{0, 5, 6, 7, 9, 26, 29, 32, 45, 50, 61, 64, 65},
 {1, 7, 17, 22, 24, 48},
 {1, 8, 23, 60},
 {0, 1, 3, 11},
 {8, 17, 33},
 {32},
 {1, 16, 22, 37},
 {5, 18, 22},
 {0, 5, 25, 28, 32, 47, 49, 52, 56},
 {35, 37},
 {38}]

In [81]:
def sets_to_lists(my_list_of_e):
    my_list_of_events_2 = []
    for i,event in enumerate(my_list_of_e):    
        x = sorted(list(event))
        my_list_of_events_2.append(x)
    print(my_list_of_events_2)

In [32]:
sets_to_lists(my_list_of_events)

[[5, 6, 7, 9, 18, 26, 27, 29, 32, 45, 50, 57, 61, 64, 65], [1, 7, 17, 22, 24, 48], [0, 1, 3, 11], [8, 17, 33], [2, 32], [1, 16, 37], [18, 22], [6, 22, 28, 32, 47, 49, 52, 56], [35, 37], [38], [22, 31, 39, 43, 47, 51], [36, 41, 58, 63]]


In [82]:
sets_to_lists(my_list_of_events)

[[0, 5, 6, 7, 9, 26, 29, 32, 45, 50, 61, 64, 65], [1, 7, 17, 22, 24, 48], [1, 8, 23, 60], [0, 1, 3, 11], [8, 17, 33], [32], [1, 16, 22, 37], [5, 18, 22], [0, 5, 25, 28, 32, 47, 49, 52, 56], [35, 37], [38]]


In [33]:
text , annotated_story = open_and_annotate('buddha_free_bird')

2021-04-18 17:08:31 INFO: Writing properties to tmp file: corenlp_server-034b5bda5077415a.props
2021-04-18 17:08:31 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-034b5bda5077415a.props -annotators tokenize,ssplit -preload -outputFormat serialized


In [34]:
locations_dict, location_number_map, total_sentences = events_by_location_and_time(text, annotated_story)
construct_event_list(locations_dict, [])

{21}

In [ ]:
[3,6,10,17,21]

In [48]:
for time_size in range(300,1100,100):
    create_lexicons(30,14,300,time_size)
    my_list_of_events = []
    for i,name in enumerate(story_names):
        text , annotated_story = open_and_annotate(name)
        locations_dict, location_number_map, total_sentences = events_by_location_and_time(text, annotated_story)
        my_list_of_events.append(construct_event_list(locations_dict, no_splitting[i]))
    print("For time_size = ", time_size)
    sets_to_lists(my_list_of_events)

2021-03-30 14:17:36 INFO: Writing properties to tmp file: corenlp_server-e2c6defa91634ff1.props
2021-03-30 14:17:36 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-e2c6defa91634ff1.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-03-30 14:17:41 INFO: Writing properties to tmp file: corenlp_server-e45431bd89bc41dc.props
2021-03-30 14:17:41 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-e45431bd89bc41dc.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-03-30 14:17:44 INFO: Writing properties to tmp file: corenlp_server-65d12f71bd674582.props
2021-03-30 14:17:44 INFO: 

For time_size =  300
[[5, 6, 7, 9, 18, 26, 29, 32, 45, 50, 61, 64, 65], [1, 7, 17, 22, 24, 48], [8, 23, 60], [0, 1, 3, 11]]


2021-03-30 14:17:57 INFO: Writing properties to tmp file: corenlp_server-43f3644be4f647e0.props
2021-03-30 14:17:57 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-43f3644be4f647e0.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-03-30 14:18:01 INFO: Writing properties to tmp file: corenlp_server-5d598c6ad96a47d6.props
2021-03-30 14:18:01 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-5d598c6ad96a47d6.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-03-30 14:18:05 INFO: Writing properties to tmp file: corenlp_server-3cc0e06c006949b8.props
2021-03-30 14:18:05 INFO: 

For time_size =  400
[[5, 6, 7, 9, 18, 26, 27, 29, 32, 45, 50, 57, 61, 64, 65], [1, 7, 17, 22, 24, 48], [8, 23, 60], [0, 1, 3, 11]]


2021-03-30 14:18:18 INFO: Writing properties to tmp file: corenlp_server-be0187e9e3ed459f.props
2021-03-30 14:18:18 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-be0187e9e3ed459f.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-03-30 14:18:22 INFO: Writing properties to tmp file: corenlp_server-da08a1cd72e34fc4.props
2021-03-30 14:18:22 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-da08a1cd72e34fc4.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-03-30 14:18:26 INFO: Writing properties to tmp file: corenlp_server-609cfc3d09fc46e3.props
2021-03-30 14:18:26 INFO: 

...

For time_size =  800
[[0, 5, 6, 7, 9, 18, 26, 27, 29, 32, 40, 45, 50, 57, 61, 64, 65], [1, 7, 17, 21, 22, 24, 27, 48], [0, 1, 8, 23, 32, 60], [0, 1, 3, 11]]


2021-03-30 14:19:38 INFO: Writing properties to tmp file: corenlp_server-374948bda6934f6a.props
2021-03-30 14:19:38 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-374948bda6934f6a.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-03-30 14:19:42 INFO: Writing properties to tmp file: corenlp_server-f555caa1506b41a6.props
2021-03-30 14:19:42 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-f555caa1506b41a6.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-03-30 14:19:46 INFO: Writing properties to tmp file: corenlp_server-99b7a62703ba4a8c.props
2021-03-30 14:19:46 INFO: 

For time_size =  900
[[0, 5, 6, 7, 9, 18, 26, 27, 29, 32, 40, 45, 50, 57, 61, 64, 65], [1, 7, 17, 21, 22, 24, 27, 48], [0, 1, 8, 23, 32, 60], [0, 1, 3, 11]]


2021-03-30 14:19:59 INFO: Writing properties to tmp file: corenlp_server-221d8f3b67154c75.props
2021-03-30 14:19:59 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-221d8f3b67154c75.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-03-30 14:20:03 INFO: Writing properties to tmp file: corenlp_server-21085e92e3534528.props
2021-03-30 14:20:03 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Giri\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-21085e92e3534528.props -annotators tokenize,ssplit -preload -outputFormat serialized
2021-03-30 14:20:07 INFO: Writing properties to tmp file: corenlp_server-f36715921ee84b18.props
2021-03-30 14:20:07 INFO: 

For time_size =  1000
[[0, 5, 6, 7, 9, 18, 26, 27, 29, 32, 40, 45, 50, 57, 61, 64, 65], [1, 7, 17, 21, 22, 24, 27, 48], [0, 1, 8, 23, 32, 60], [0, 1, 3, 11]]


In [ ]:
events = []
for sn in range(len(story_names)):
    text , annotated_story, sen_char_map = open_and_annotate(story_names[sn])
    no_splitting_here = []
    flag=0
    for i, sentence in enumerate(annotated_story.sentence):
        sentence = text[sentence.characterOffsetBegin:sentence.characterOffsetEnd].strip("\n").replace(", "," ").replace(".","").replace("-"," ").replace("?","").replace("!","").replace(":"," ").replace("\n\n","").replace("\n \n", '')
        no_of_quotes = sentence.count('"')
        if no_of_quotes%2 != 0:        
            flag = (flag + 1)%2
        if flag != 0 or (no_of_quotes!=0):
            no_splitting_here.append(i)
    print(story_names[sn])
    events.append(HAC(text, annotated_story, sen_char_map, no_splitting_here))
    print("\n\n\n")